In [6]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

In [19]:
ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [27]:
conn.rollback()
import pandas as pd

# 날짜 범위 생성
periods = []
for year in range(2015, 2025):
    # 상반기
    periods.append({
        'name': f'{year} 상반기',
        'start': f'{year}-01-01',
        'end': f'{year}-07-01'
    })
    # 하반기
    periods.append({
        'name': f'{year} 하반기',
        'start': f'{year}-07-01',
        'end': f'{year+1}-01-01'
    })

# 결과를 저장할 리스트
all_results = []

# 각 기간별 데이터 조회
for period in periods:
    qry = f"""
        SELECT 
            COUNT(ch."createdAt") as total_count,
            SUM(CASE WHEN ch."status" = 'lost' THEN 1 ELSE 0 END) as lost_count
        FROM credit_history ch
        WHERE ch."openedAt" >= $${period['start']}$$ 
        AND ch."openedAt" < $${period['end']}$$
        """
    cursor.execute(qry)
    result = cursor.fetchone()
    
    # 결과를 리스트에 추가
    all_results.append({
        '기간': period['name'],
        '총 대출 수': result[0],
        '부실 대출 수 (lost)': result[1]
    })

print(all_results)
cursor.execute(qry)
result = cursor.fetchall()


base_filepath = './'
# DataFrame 생성 및 CSV 저장
df = pd.DataFrame(all_results, columns=['기간', '총 대출 수', '부실 대출 수 (lost)'])
df.to_csv(base_filepath + '/반기별_대출현황.csv', index=False, encoding='utf-8-sig')


[{'기간': '2015 상반기', '총 대출 수': 4, '부실 대출 수 (lost)': 1}, {'기간': '2015 하반기', '총 대출 수': 9, '부실 대출 수 (lost)': 0}, {'기간': '2016 상반기', '총 대출 수': 11, '부실 대출 수 (lost)': 2}, {'기간': '2016 하반기', '총 대출 수': 23, '부실 대출 수 (lost)': 6}, {'기간': '2017 상반기', '총 대출 수': 19, '부실 대출 수 (lost)': 1}, {'기간': '2017 하반기', '총 대출 수': 50, '부실 대출 수 (lost)': 4}, {'기간': '2018 상반기', '총 대출 수': 96, '부실 대출 수 (lost)': 16}, {'기간': '2018 하반기', '총 대출 수': 127, '부실 대출 수 (lost)': 24}, {'기간': '2019 상반기', '총 대출 수': 214, '부실 대출 수 (lost)': 28}, {'기간': '2019 하반기', '총 대출 수': 315, '부실 대출 수 (lost)': 36}, {'기간': '2020 상반기', '총 대출 수': 257, '부실 대출 수 (lost)': 41}, {'기간': '2020 하반기', '총 대출 수': 350, '부실 대출 수 (lost)': 59}, {'기간': '2021 상반기', '총 대출 수': 409, '부실 대출 수 (lost)': 76}, {'기간': '2021 하반기', '총 대출 수': 377, '부실 대출 수 (lost)': 109}, {'기간': '2022 상반기', '총 대출 수': 324, '부실 대출 수 (lost)': 63}, {'기간': '2022 하반기', '총 대출 수': 327, '부실 대출 수 (lost)': 67}, {'기간': '2023 상반기', '총 대출 수': 320, '부실 대출 수 (lost)': 80}, {'기간': '2023 하반기', '총 대출 수': 289, '부실 대출 수 (